In [1]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
con = sqlite3.connect("D:/Data_Practice/final.sqlite")

processed_data = pd.read_sql_query('''SELECT * FROM Reviews''',con)
processed_data.shape
processed_data.head()

final_pos = processed_data[processed_data.Score=='Positive']
final_pos.shape

final_neg = processed_data[processed_data.Score=='Negative']
final_neg.shape

final_pos_5000 = final_pos[0:5000]
final_pos_5000.shape
print(final_pos_5000.head(5))

final_neg_5000 = final_neg[0:5000]
final_neg_5000.shape
print(final_neg_5000.head(5))

final_10000 = pd.concat([final_pos_5000 , final_neg_5000],axis=0,ignore_index=True)
final_10000.tail()

sample = final_10000.sort_values(by='Time')
score_10000 = sample['Score']
sample.head()
sample['Score'].value_counts()

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(1,1))
final_bow = count_vect.fit_transform(sample['Cleaned_Text'].values)
print(final_bow.shape)
#Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
std_data = scaler.fit_transform(final_bow.todense())

    index      Id   ProductId          UserId                  ProfileName  \
0  138706  150524  0006641040   ACITT7DI6IDDL              shari zychinski   
1  138688  150506  0006641040  A2IW4PEEKO2R0U                        Tracy   
2  138689  150507  0006641040  A1S4A3IQ2MU7V4        sally sue "sally sue"   
3  138690  150508  0006641040     AZGXZ2UUK6X  Catherine Hallberg "(Kate)"   
4  138691  150509  0006641040  A3CMRKGE0P909G                       Teresa   

   HelpfulnessNumerator  HelpfulnessDenominator     Score        Time  \
0                     0                       0  Positive   939340800   
1                     1                       1  Positive  1194739200   
2                     1                       1  Positive  1191456000   
3                     1                       1  Positive  1076025600   
4                     3                       4  Positive  1018396800   

                                      Summary  \
0                   EVERY book is education

In [6]:
sample['Score'].value_counts()

Positive    5000
Negative    5000
Name: Score, dtype: int64

In [5]:
#Dim Red is optional
from sklearn.manifold import TSNE
model = TSNE(n_components=3,random_state=0)
tsne_data = model.fit_transform(final_dense)
tsne_data.shape

ValueError: 'n_components' should be inferior to 4 for the barnes_hut algorithm as it relies on quad-tree or oct-tree.

In [14]:
type(std_data)
score_1000.shape

(1000,)

In [18]:
std_1000 = std_data[0:10000]
score_1000 = score_10000[0:9000]

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import *
from sklearn.linear_model import LogisticRegression

#data = load_breast cancer() refer: http://scikit-learn.org/stable/modules/

tuned_parameters = [{'C': [10**-4, 10**-2, 10, 100, 10**3]}]

X = std_data #We give entire dim without tSNE here
Y = score_10000

x_1,x_test,y_1,y_test = train_test_split(X,Y,test_size=0.3,random_state=10)#70 - 30
#x_tr,x_cv,y_tr,y_cv   = train_test_split(x_1,y_1,test_size=0.3,random_state=10) # 70% of 70-->x_tr;30% of 70 -->x_cv/y_cv
#x_tr = 73,x_cv=32,x_test=45 and same y's also

#Using GridSearchcv

model = GridSearchCV(LogisticRegression(), tuned_parameters, scoring = 'accuracy')

model.fit (x_1, y_1)

print(model.best_estimator_) 
print (model.score(x_1, y_1)) 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

LogisticRegression(C=0.0001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
0.9627142857142857


In [38]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
clf = LogisticRegression(C=0.0001,penalty='l2')
clf.fit(x_1,y_1)

pred = clf.predict(x_test)
 
acc = accuracy_score(y_test,pred)*100
print(acc)


86.26666666666667
